# Setup
## Imports

In [ ]:
from ultralytics import YOLO
import os
import shutil
import random

## Shuffling in Validation

In [ ]:
# Get train images and labels
train = os.listdir('../data/hourglass_localization_rectification/yolo_failures/train/images')

# Move random 20% of them to validation set
val = random.sample(train, int(len(train) * 0.2))
for file in val:
    # Move image
    src = os.path.join('../data/hourglass_localization_rectification/yolo_failures/train/images', file)
    dst = os.path.join('../data/hourglass_localization_rectification/yolo_failures/val/images', file)
    shutil.move(src, dst)

    # Move label
    src = os.path.join('../data/hourglass_localization_rectification/yolo_failures/train/labels', file.replace('.jpg', '.txt'))
    dst = os.path.join('../data/hourglass_localization_rectification/yolo_failures/val/labels', file.replace('.jpg', '.txt'))
    shutil.move(src, dst)

# Training

In [ ]:
# Find latest "trainXX" folder
model_to_train = [d.replace('train', '') for d in os.listdir('../yolo/runs/obb') if d.startswith('train')]
model_to_train.remove('') # remove empty string
model_to_train = sorted(model_to_train, key=int)[-1] # get the latest one
print(f'Training from train{model_to_train}')

model = YOLO(f'../yolo/runs/obb/train{model_to_train}/weights/best.pt') # load best model from training

In [ ]:
results = model.train(
    data='../data/hourglass_localization_rectification/yolo_failures/data.yaml',
    time=24,                              # Max training time in hours
    imgsz=640,                            # Image size for training (default for YOLO is 640x640)
    patience=10,                          # Early stopping patience (after this many epochs with no improvement stop training)
    pretrained=False,                      # Don't pre-trained weights (use the latest model)
    plots=True,                           # Create plots

    workers=8,                            # Number of worker threads for data loading
    batch = 16,                           # Batch size
)